# 使用FP-growth算法来发现频繁项集
---
使用 FP-growth算法更有效的挖掘`频繁项集`, 它只需要对数据库进行两次扫描, 而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁.
FP-growth算法将数据存储在一种称为FP树的紧凑数据结构中.FP代表频繁模式(Frequent Pattern)

使用实例: 搜索引擎自动补全查询词项.

基本过程:
- 构建FP树
- 从FP树中挖掘频繁项集

In [1]:
class TreeNode:
    def __init__(self, name, counts, parent):
        self.name = name  # 节点名称
        self.counts = counts  # 节点出现次数
        self.parent = parent  # 父节点
        self.children = {}  # 子节点
        # 节点链接
        self.node_link = None  # 用于连接 不同项集 的 相同的项
    
    def inc(self, counts):
        """
        增加项出现的次数
        """
        self.counts += counts
    
    def disp(self, index=1):
        """
        文本方式显示树
        """
        print(' ' * index, self.name, ' ', self.counts)

## FP-growth 原理
[构建FP树](https://github.com/apachecn/AiLearning/blob/master/docs/ml/12.%E4%BD%BF%E7%94%A8FP-growth%E7%AE%97%E6%B3%95%E6%9D%A5%E9%AB%98%E6%95%88%E5%8F%91%E7%8E%B0%E9%A2%91%E7%B9%81%E9%A1%B9%E9%9B%86.md#fp-growth-%E5%8E%9F%E7%90%86)

In [4]:
def load_data():
    sample = [
        ['r', 'z', 'h', 'j', 'p'],
       ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
       ['z'],
       ['r', 'x', 'n', 'o', 's'],
    #    ['r', 'x', 'n', 'o', 's'],
       ['y', 'r', 'x', 'z', 'q', 't', 'p'],
       ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']
    ]
    return sample

In [8]:
data = load_data()

In [42]:
class FPTree:
    def __init__(self, min_support = 1):
        # 出现的最少次数 低于此值的项会被丢弃
        self.min_support = min_support
    
    def fit(self, data):
        data = self.init_data(data)
        
        head_table = {}
        # 1. 遍历所有的数据集合，计算所有项的支持度。
        for set_ in data:
            for item in set_:
                # set_ : frozenset(['z', 'y', 'x', 'w', 'v', 'u', 't', 's'])
                # 'z': 1(z累计次数) + 1(当前set出现次数) 
                head_table[item] = head_table.get(item, 0) + data[set_]
       
        # 2. 丢弃非频繁的项
        for key in list(head_table.keys()):  # 需要创建一个新的list
            if head_table[key] < self.min_support:
                del head_table[key]
#         print(head_table)
        if not head_table:
            return None, None
        freq_sets = set(head_table.keys())
        # 3. 基于 支持度 降序排序所有的项。
        sorted_list = sorted(head_table.items(), key=lambda item: item[1], reverse=True)
        # 4. 所有数据集合按照得到的顺序重新整理
        # 5. 重新整理完成后，丢弃每个集合末尾非频繁的项。 
        print(sorted_list)
        new_data = {}
        for set_, freqs in data.items():
            
            key = frozenset([item[0] for item in sorted_list if item[0] in set_][::-1])
            new_data[key] = freqs
        print(new_data)
    
    def init_data(self, data):
        ret_dict = {}  # {frozenset() : 次数}
        for item in data:
            ret_dict[frozenset(item)] = ret_dict.get(frozenset(item), 0) + 1
        return ret_dict

In [43]:
fp = FPTree(min_support=3)
fp.fit(data)

[('z', 5), ('x', 4), ('r', 3), ('t', 3), ('y', 3), ('s', 3)]
{frozenset({'r', 'z'}): 1, frozenset({'t', 'y', 'x', 's', 'z'}): 1, frozenset({'z'}): 1, frozenset({'x', 'r', 's'}): 1, frozenset({'t', 'y', 'x', 'r', 'z'}): 1}


In [27]:
dict_ = {'r': 3, 'z': 5, 't': 3, 'y': 3, 'x': 4, 's': 3}

sorted(dict_.items(), key=lambda item: item[1], reverse=True)

[('z', 5), ('x', 4), ('r', 3), ('t', 3), ('y', 3), ('s', 3)]

In [39]:
{'z', 'r', 'h', 'j', 'p'} & frozenset(['z', 'j', 'x', 'r'])

{'j', 'r', 'z'}